## Basic Data Using Scrapetube:

In [1]:
# !pip install scrapetube 

In [2]:
# Imports
import numpy as np
import pandas as pd
import scrapetube
from datetime import datetime

# Parameters
max_entries = None

### Get basic data:

For testing, use a low number of max_entries. To remove the limit, use `max_entries = None`.

In [3]:
GMM_url = "https://www.youtube.com/@GoodMythicalMorning"
video_iterator = scrapetube.get_channel(channel_url=GMM_url, limit=max_entries, sort_by="newest")

# New dictionary class with multidimensional get
class custom_dict(dict):
    def multidim_get(self, *keys):
        """
        Allows .get() method to operate on nested dictionarys.
        """
        value = self
        for key in keys:
            try:
                value = value[key]
            except KeyError:
                return None
        return value

# Features to extract from Scrapetube video object
"""
#ID
    - name
    - length
    - views
    - published date
    - thumbnail
        - still
        - video
    - scrape datetime
"""

# Function to extract features
def get_basic_video_details(video):
    video = custom_dict(video)
    return {
        "id": video.multidim_get("videoId"),
        "name": video.multidim_get("title","runs",0,"text"),
        "duration": video.multidim_get("lengthText","simpleText"),
        "views": video.multidim_get("viewCountText","simpleText"),
        "published": video.multidim_get("publishedTimeText","simpleText"),
        "thumbnail": {
            "still": video.multidim_get("thumbnail","thumbnails",-1,"url"),
            "video": video.multidim_get("richThumbnail","movingThumbnailRenderer","movingThumbnailDetails","thumbnails",0,"url")
        },
        "scraped": datetime.now()
    }

# Build a dataframe of episodes using our Scrapetube iterator
df = pd.DataFrame([
    get_basic_video_details(video)
    for video
    in video_iterator
])

### Clean duration values:
Standardize format to "HH:MM:SS" then convert to integer value (seconds)

In [4]:
def _leading_timecode(string, timecode_format="00:00:00"):
    """
    Standardize timecode with leading zeros and delimiters
    """
    return timecode_format[:len(string)-1:-1] + string

df["duration"] = np.dot(
    df["duration"].apply(_leading_timecode).str.split(":", expand=True).astype(int), # Get hours, minutes, and seconds
    [3600, 60, 1] # Multiply by 3x1 matrix to convert to total seconds
    )

### Clean view counts:
Convert from format "##,###,### views" to integer value (views)

In [5]:
df["views"] = df["views"].str.replace("\D", "", regex=True).astype(int, errors="ignore") # Remove any non-integer characters then convert to int

### Check output:
This is as far as we can get with ScrapeTube. However, we can still get more information about these videos using the YouTube Data API!

In [6]:
df.head()

,id,name,duration,views,published,thumbnail,scraped
0,fsJU9mOcvhQ,Our Most Unhinged Moments This Year,1322,493306,1 day ago,{'still': 'https://i.ytimg.com/vi/fsJU9mOcvhQ/...,2022-12-22 23:57:51.517487
1,XiORNYGT-6s,Our Best Food Creations This Year,1331,936835,3 days ago,{'still': 'https://i.ytimg.com/vi/XiORNYGT-6s/...,2022-12-22 23:57:51.517504
2,B6dXVr0r0Ws,We Tried EVERY Goldfish Flavor,1194,1538449,6 days ago,{'still': 'https://i.ytimg.com/vi/B6dXVr0r0Ws/...,2022-12-22 23:57:51.517511
3,RPp5CXZVhlc,We Hug For 20 Minutes Straight... For Science,1394,550021,7 days ago,{'still': 'https://i.ytimg.com/vi/RPp5CXZVhlc/...,2022-12-22 23:57:51.517520
4,JrZP8aAZE9M,Lab Grown Dairy Taste Test,1140,920359,8 days ago,{'still': 'https://i.ytimg.com/vi/JrZP8aAZE9M/...,2022-12-22 23:57:51.517529


### Export to CSV:

In [7]:
output_path = "data/gmm-episodes_basic.csv"
df.to_csv(output_path, index=False)

---

## More data with YouTube Data API:

In [8]:
# !pip install isodate

In [9]:
# Imports
import isodate
import json
import requests
from zipfile import ZipFile

# Parameters
basic_dataset = "data/gmm-episodes_basic.csv"
max_search_pages = 5 # Will not be used if you supply a basic dataset to supply video IDs

### Create a Google Cloud project:

YouTube Data API calls are limited by a daily quota, we need to run our calls through a Google Cloud project in order to keep track of our quota usage.

**Steps:**
1. Create a Google Cloud project [here](https://console.cloud.google.com/).
2. Enable the [YouTube Data API](https://developers.google.com/youtube/v3) for your project.

### Create and import an API Key:

To run API calls we will need a Key to link this notebook to our Google Cloud project.

**Steps:**
1. Create an API Key [here](https://console.cloud.google.com/apis/credentials) for your project.
2. Create a file to store this API Key using the `credentials-template.json` template. Name the new file `credentials.json`.

In [10]:
credentials_file = "credentials.json"

with open(credentials_file, "r") as fh:
    credentials = json.load(fh)

assert "API_Key" in list(credentials.keys())
assert isinstance(credentials["API_Key"], str)

print("Successfully loaded API key from credentials file.")

Successfully loaded API key from credentials file.


### Get all videos from channel:

We can use the search API to build a list of all videos on the Good Mythical Morning channel, however, we are limited to retrieving 50 videos at a time, so we will need to break this process up into several queries. To do this, we will retrieve one page at a time and then combine our results. 

For testing, use a low number of pages. To remove this limit, set `max_pages` to an arbitrarily large value (like infinity). 

In [11]:
def dict_merge(base, *args):
    """
    Helper function for merging n dictionaries.
    """
    for dictionary in args: base |= dictionary
    return base


def get_channel_videos_page(channel_id, page_token=None, credential=credentials["API_Key"], max_page_results=50, order="date", parts=["id"]):
    """
    Fucntion to retrieve one page of videos.

    Parameters:
    channel_id - the ID of the channel to get videos from
    page_token - the ID of the page we are looking for. if no page specified this should be 'None'
    credential - the API Key used for the query
    max_page_results - the number of results to return in each page. this should be limited to 50
    order - how to sort the videos we are returning
    parts - the pieces of information to retrieve in our query
    """

    url = "https://www.googleapis.com/youtube/v3/search" +\
        f"?key={credential}" +\
        f"&channelId={channel_id}" +\
        f"&maxResults={max_page_results}" +\
        f"&order={order}" +\
        f"&part={','.join(parts)}" +\
        f"{f'&pageToken={page_token}' if page_token else ''}"

    webpage = requests.get(url)
    content = json.loads(webpage.text)

    next_page = content.get("nextPageToken")
    videos = content.get("items")

    if not videos: raise Exception(content["error"]["message"])
    
    videos_data = [
        dict_merge(*(video[part] for part in parts))
        for video
        in videos
        if video["id"]["kind"] == "youtube#video"
    ]

    return videos_data, next_page


def get_channel_videos(channel_id, max_depth=10, **kwargs):
    """
    Function to retrieve all videos from a channel, where count is greater than can be retrieved in a single page.

    Parameters:
    channel_id - the ID of the channel to get videos from
    max_depth - the maximum number of pages to query
    **kwargs - arguments to pass to get_channel_videos_page
    """

    all_videos = []
    
    page_videos, next_page = get_channel_videos_page(channel_id, **kwargs)
    all_videos += page_videos

    depth = 1
    while next_page and depth < max_depth:
        page_videos, next_page = get_channel_videos_page(channel_id, page_token=next_page, **kwargs)
        all_videos += page_videos
        depth += 1

    return all_videos

if basic_dataset:
    video_ids = pd.read_csv(basic_dataset)["id"].to_list()
    videos = [{"videoId": id, "kind": "youtube#video"} for id in video_ids]
else:
    GMM_channel_id = "UC4PooiX37Pld1T8J5SYT-SQ"
    videos = get_channel_videos(GMM_channel_id, max_depth=max_search_pages)

### Get more data from YouTube DataAPI

In [12]:
def get_video_data(video_id, parts=["snippet", "statistics", "contentDetails"], credential=credentials["API_Key"]):
    """
    Using the YouTube Data API, get information for a video by it's video ID.
    
    Parameters:
    video_id - the ID of the video to find
    parts - the parts of data to return in API call. documentation here https://developers.google.com/youtube/v3/getting-started#partial
    credential - the API Key to use for the call
    """
    
    url = "https://www.googleapis.com/youtube/v3/videos" +\
        f"?key={credential}" +\
        f"&part={','.join(parts)}" +\
        f"&id={video_id}"

    webpage = requests.get(url)
    content = json.loads(webpage.text)

    video = content["items"][0]

    return dict_merge(
        {"id": video_id},
        {"scraped": pd.to_datetime(str(datetime.utcnow().replace(microsecond=0))+"+00:00")},
        *(video[part] for part in parts)
        )


for entry in videos:
    entry |= get_video_data(entry["videoId"])

df = pd.DataFrame(videos)
df = df.drop(columns=["kind", "videoId", "channelId", "channelTitle", "localized"]) # Remove redundant columns 

### Process time formats

`publishedAt` and `duration` use a specific format. We can convert the `publishedAt` to date time, and the `duration` to seconds.

In [13]:
df["duration"] = df["duration"].apply(lambda duration: isodate.parse_duration(duration).seconds)
df["publishedAt"] = df["publishedAt"].apply(isodate.parse_datetime)

### Remove uninformative columns

Some columns have only one unique value, there is no point keeping these, so we can drop them. If you would like to keep them, feel free to skip this block.

In [14]:
for column in df.columns:
    if df[column].map(str).nunique() == 1:
        print(f"Dropping column {column}. The single value in column was: '{df[column].loc[0]}'")
        df.drop(column,axis=1,inplace=True)

Dropping column liveBroadcastContent. The single value in column was: 'none'
Dropping column favoriteCount. The single value in column was: '0'
Dropping column dimension. The single value in column was: '2d'
Dropping column projection. The single value in column was: 'rectangular'


Some columns may not be useful for other reasons, such as having some variation, but not enough to be meaningful. We can drop these too. If you disagree with the reasoning below, feel free to modify the list of columns we've chosen to drop.
- `defaultLanguage` and `defaultAudioLanguage`:  
    - *All videos are in English. Even though this is encoded in different ways sometimes, all values have the same meaning, so we can drop these features.*
- `categoryId`:  
    - *All videos since 2014 have been marked as category 24 ("entertainment"), therefore, for most purposes, this feature will be unreliable.*
- `definition`:
    - *Most videos since 2014 have been uploaded in HD, therefore, for most purposes, this feature will be unreliable.*
- `caption`:
    - *Nearly all videos since 2016 have captions, therefore, for most purposes, this feature will be unreliable.*
- `licensedContent`:
    - *While a few episodes have `False` values for this feature, there is not nearly enough to be meaningful.*
- `regionRestriction`:
    - *Only one video has a region restriction (Buddy System Ep. 3, which is blocked in Latvia). We can remove this column, as there isn't enough variation to derive any innsight.*

In [15]:
uninformative_columns = ["defaultLanguage", "defaultAudioLanguage", "categoryId", "definition", "caption", "licensedContent", "regionRestriction"]

df.drop(columns=uninformative_columns, inplace=True)

### Transform content rating column

The content of the values in the `contentRating` column aren't overly useful. To make the data more readable we can convert this feature to a binary one (`ageRestricted`).

In [16]:
df["ageRestricted"] = df["contentRating"] != {}
df.drop(columns="contentRating", inplace=True)

### Preview data

After completed our transformations and dropping features, we can preview our dataset.

In [17]:
df.head()

,id,scraped,publishedAt,title,description,thumbnails,tags,viewCount,likeCount,commentCount,duration,ageRestricted
0,fsJU9mOcvhQ,2022-12-23 05:59:54+00:00,2022-12-21 11:00:33+00:00,Our Most Unhinged Moments This Year,"Today, we're looking back at our most unhinged...",{'default': {'url': 'https://i.ytimg.com/vi/fs...,"[gmm, good mythical morning, rhettandlink, rhe...",493306,19553,742,1322,False
1,XiORNYGT-6s,2022-12-23 05:59:54+00:00,2022-12-19 11:00:15+00:00,Our Best Food Creations This Year,"Today, we're looking back at our favorite food...",{'default': {'url': 'https://i.ytimg.com/vi/Xi...,"[gmm, good mythical morning, rhettandlink, rhe...",936835,30037,1063,1331,False
2,B6dXVr0r0Ws,2022-12-23 05:59:54+00:00,2022-12-16 11:00:06+00:00,We Tried EVERY Goldfish Flavor,"Today, we're eating way too many Goldfish! GMM...",{'default': {'url': 'https://i.ytimg.com/vi/B6...,"[gmm, good mythical morning, rhettandlink, rhe...",1538547,50280,1821,1194,False
3,RPp5CXZVhlc,2022-12-23 05:59:54+00:00,2022-12-15 11:00:12+00:00,We Hug For 20 Minutes Straight... For Science,"Today, we're joined by Noah Centineo to learn ...",{'default': {'url': 'https://i.ytimg.com/vi/RP...,"[gmm, good mythical morning, rhettandlink, rhe...",550021,23600,1405,1394,False
4,JrZP8aAZE9M,2022-12-23 05:59:55+00:00,2022-12-14 11:00:09+00:00,Lab Grown Dairy Taste Test,"Today, we're seeing if we can taste the differ...",{'default': {'url': 'https://i.ytimg.com/vi/Jr...,"[gmm, good mythical morning, rhettandlink, rhe...",920359,29636,1527,1140,False


### Store large text columns in an archive

While we can store our large text objects, like video descriptions directly in our Pandas DataFrame, this gets quite messy when we try to save our data as a CSV. Instead, we will store each description in it's own text file.

In [18]:
archive_columns = ["description", "thumbnails"]

zip_path = "data/gmm_episodes_files.zip"
zip = ZipFile(zip_path, "w")


def _store_in_archive(archive, name, content, folder=None, extension = ".txt"):
    """
    Write string to file in an archive
    """
    path = name + extension
    if folder: path = folder + "/" + path
    archive.writestr(path, str(content))


def row_to_archive(df, archive, name_column, content_column, **kwargs):
    """
    Store a column of a DataFrame as a folder of files in an archive
    """
    df.apply(lambda row: _store_in_archive(
        archive=archive,
        name=row[name_column],
        content=row[content_column],
        folder=content_column,
        **kwargs
    ), axis=1)


for column in archive_columns:
    row_to_archive(df, zip, name_column="id", content_column=column)

zip.close()

### Export to CSV

In [19]:
output_path = "data/gmm-episodes_full.csv"
df.drop(columns=archive_columns).to_csv(output_path, index=False)